In [1]:
from DataParser import DataParser
import numpy as np
import scipy.sparse as sps
from sklearn.model_selection import KFold

from Data_manager.split_functions.split_train_validation_random_holdout import \
split_train_in_two_percentage_global_sample

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from SimpleEvaluator import evaluator
from Base.Evaluation.Evaluator import EvaluatorHoldout

In [2]:
from Base.NonPersonalizedRecommender import TopPop, Random
from Hybrid.HybridCombinationSearchCVtest import HybridCombinationSearchCVtest
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from MatrixFactorization.IALSRecommender import IALSRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from SLIM_ElasticNet.SSLIM_ElasticNet import SSLIMElasticNet
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from MatrixFactorization.NMFRecommender import NMFRecommender
from KNN.ItemKNN_CBF_CF import ItemKNN_CBF_CF

In [3]:
seed = 1666
parser = DataParser()

URM_all = parser.get_URM_all()
ICM_all = parser.get_ICM_all()

#URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85, seed=seed)

In [4]:
rp3b = (RP3betaRecommender, {'topK': 121, 'alpha': 0.20195468746532336, 'beta': 0.004377103282444673, 'normalize_similarity': False})
p3a = (P3alphaRecommender, {'topK': 344, 'alpha': 0.4233795425391197, 'normalize_similarity': False})
icf = (ItemKNNCFRecommender, {'topK': 994, 'shrink': 981, 'similarity': 'asymmetric', 'normalize': True, 'asymmetric_alpha': 0.05110465631417439, 'feature_weighting': 'TF-IDF'})
ucf = (UserKNNCFRecommender, {'topK': 342, 'shrink': 1, 'similarity': 'cosine', 'normalize': True, 'feature_weighting': 'none'})
icb = (ItemKNNCBFRecommender, {'topK': 22, 'shrink': 59, 'similarity': 'dice', 'normalize': False})
sbpr = (SLIM_BPR_Cython, {'topK': 989, 'epochs': 90, 'symmetric': False, 'sgd_mode': 'adam', 'lambda_i': 1.7432198130463203e-05, 'lambda_j': 0.0016819750046109673, 'learning_rate': 0.00031293205801039345})
icfcb = (ItemKNN_CBF_CF, {'topK': 651, 'shrink': 858, 'similarity': 'asymmetric', 'normalize': True, 'asymmetric_alpha': 0.01038549339003936, 'feature_weighting': 'TF-IDF', 'icm_weight': 1.0091171832447683})
#sen = (SLIMElasticNetRecommender, {'topK': 992, 'l1_ratio': 0.004065081925341167, 'alpha': 0.003725005053334143})
#sslim = (SSLIMElasticNet, {'beta': 0.4849594591575789, 'topK': 1000, 'l1_ratio': 1e-05, 'alpha': 0.001})

In [5]:
combo_labels ={
    'combo1': 'icb_icf_p3a',
    'combo2': 'icb_icf_rp3b',
    'combo3': 'icb_icf_ucf',
    'combo4': 'icb_rp3b_p3a',
    'combo5': 'icb_ucf_p3a',
    'combo6': 'icb_ucf_rp3b',
    'combo7': 'icf_ucf_p3a',
    'combo8': 'icf_ucf_rp3b',
    'combo9': 'p3a_icfcb_sbpr',
    'combo10': 'rp3b_icfcb_sbpr',
    'combo11': 'rp3b_p3a_icfcb',
    'combo12': 'rp3b_p3a_sbpr',
    'combo13': 'ucf_icfcb_sbpr',
    'combo14': 'ucf_p3a_icfcb',
    'combo15': 'ucf_p3a_sbpr',
    'combo16': 'ucf_rp3b_icfcb',
    'combo17': 'ucf_rp3b_p3a',
    'combo18': 'ucf_rp3b_sbpr',
    
    'combo19': 'icb_icf_icfcb',
    'combo20': 'icb_icf_sbpr',
    'combo21': 'icb_p3a_icfcb',
    'combo22': 'icb_rp3b_icfcb',
    'combo23': 'icb_rp3b_sbpr',
    'combo24': 'icb_ucf_icfcb',
    #'combo25': 'icb_ucf_sbpr',
    'combo26': 'icb_p3a_sbpr',
    'combo27': 'icb_icfcb_sbpr',
    'combo28': '',
}

In [6]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=seed)

URM_list = []
URM_test_list = []
evaluator_list = []

shape = URM_all.shape
indptr = URM_all.indptr
indices = URM_all.indices
data = URM_all.data

for train_index, test_index in kf.split(data):
    data_train = np.ones(data.shape)
    data_test = np.ones(data.shape)
    data_train[test_index] = 0
    data_test[train_index] = 0
    kf_train = sps.csr_matrix((data_train, indices, indptr), shape=shape).copy()
    kf_test = sps.csr_matrix((data_test, indices, indptr), shape=shape).copy()
    kf_train.eliminate_zeros()
    kf_test.eliminate_zeros()
    URM_list.append(kf_train)
    URM_test_list.append(kf_test)
    evaluator_list.append(EvaluatorHoldout(kf_test, cutoff_list=[10]))

In [7]:
list_recommender = [icb, icf, p3a]
result_list = []
for i in range(len(URM_list)):
    combo1 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.6795644006301982, 'l1_ratio': 0.16897463528047377} #MAP 0.0569727
    combo1.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo1)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.72 sec. Users per second: 844
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.35 sec. Users per second: 885
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=16

In [8]:
list_recommender = [icb, icf, rp3b]
result_list = []
for i in range(len(URM_list)):
    combo2 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.17669196327123107, 'l1_ratio': 0.3076285474940705} #MAP: 0.0563723
    combo2.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo2)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.29 sec. Users per second: 901
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.25 sec. Users per second: 899
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=16

In [9]:
list_recommender = [icb, icf, ucf]
result_list = []
for i in range(len(URM_list)):
    combo3 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.6795644006301982, 'l1_ratio': 0.16897463528047377} #MAP 0.0569727
    combo3.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo3)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
UserKNNCFRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.18 sec. Users per second: 789
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
UserKNNCFRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.69 sec. Users per second: 839
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
UserKNNCFRecommender loaded. [s

In [10]:
list_recommender = [icb, rp3b, p3a]
result_list = []
for i in range(len(URM_list)):
    combo4 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.3003711591677414, 'l1_ratio': 0.520142870843707}
    combo4.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo4) #0.0568581
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.01 sec. Users per second: 943
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.03 sec. Users per second: 932
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fo

In [11]:
list_recommender = [icb, ucf, p3a]
result_list = []
for i in range(len(URM_list)):
    combo5 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.5989306837162718, 'l1_ratio': 0.19159245289609933}
    combo5.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo5) #0.0570313
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
UserKNNCFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.44 sec. Users per second: 879
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
UserKNNCFRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.04 sec. Users per second: 797
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
UserKNNCFRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=16

In [12]:
list_recommender = [icb, ucf, rp3b]
result_list = []
for i in range(len(URM_list)):
    combo6 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.22386976323299096, 'l1_ratio': 0.5121960118838208}
    combo6.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo6)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
UserKNNCFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.61 sec. Users per second: 857
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
UserKNNCFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.38 sec. Users per second: 880
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
UserKNNCFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=16

In [13]:
list_recommender = [icf, ucf, p3a]
result_list = []
for i in range(len(URM_list)):
    combo7 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.176675887233281, 'l1_ratio': 0.11869923626931214}

    combo7.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo7)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
UserKNNCFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.96 sec. Users per second: 814
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
UserKNNCFRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.13 sec. Users per second: 788
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
UserKNNCFRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666,

In [14]:
list_recommender = [icf, ucf, rp3b]
result_list = []
for i in range(len(URM_list)):
    combo8= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.961794201569176, 'l1_ratio': 0.6153004656692367}

    combo8.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo8)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
UserKNNCFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.36 sec. Users per second: 770
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
UserKNNCFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.07 sec. Users per second: 794
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
UserKNNCFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666,

In [15]:
list_recommender = [p3a, icfcb, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo9= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.9901471939440843, 'l1_ratio': 0.004332872503698072}

    combo9.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo9)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
P3alphaRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.41 sec. Users per second: 765
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
P3alphaRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.29 sec. Users per second: 770
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
P3alphaRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender l

In [16]:
list_recommender = [rp3b, icfcb, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo10= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.9929867621463604, 'l1_ratio': 0.009578827985342044}

    combo10.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo10)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 8.63 sec. Users per second: 657
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 8.03 sec. Users per second: 700
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender l

In [17]:
list_recommender = [rp3b, p3a, icfcb]
result_list = []
for i in range(len(URM_list)):
    combo11= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.05856975370672747, 'l1_ratio': 0.4436754753867742}

    combo11.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo11)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.98 sec. Users per second: 812
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.86 sec. Users per second: 819
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender loaded.

In [18]:
list_recommender = [rp3b, p3a, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo12= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.9905861627687869, 'l1_ratio': 0.0009938396703816912}

    combo12.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo12)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.67 sec. Users per second: 739
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.99 sec. Users per second: 804
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
RP3betaRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender loaded. [seed=1666, fold=

In [19]:
list_recommender = [ucf, icfcb, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo13= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.9914207032100951, 'l1_ratio': 0.007542874998078932}

    combo13.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo13)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.97 sec. Users per second: 711
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.64 sec. Users per second: 735
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recomme

In [20]:
list_recommender = [ucf, p3a, icfcb]
result_list = []
for i in range(len(URM_list)):
    combo14= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.016697522704458127, 'l1_ratio': 0.3696380999702476}

    combo14.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo14)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.64 sec. Users per second: 742
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.19 sec. Users per second: 781
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender l

In [21]:
list_recommender = [ucf, p3a, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo15= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.9933589553636074, 'l1_ratio': 0.0060490313929022505}

    combo15.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo15)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.03 sec. Users per second: 806
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.76 sec. Users per second: 831
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender loaded. [seed=1666,

In [22]:
list_recommender = [ucf, rp3b, icfcb]
result_list = []
for i in range(len(URM_list)):
    combo16 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params= {'alpha': 0.016697522704458127, 'l1_ratio': 0.3696380999702476}

    combo16.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo16)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.36 sec. Users per second: 770
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.18 sec. Users per second: 783
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommender l

In [23]:
list_recommender = [ucf, rp3b, p3a]
result_list = []
for i in range(len(URM_list)):
    combo17= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params= {'alpha': 0.014042798897581758, 'l1_ratio': 3.1039843699565146e-05}

    combo17.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo17)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 6.47 sec. Users per second: 876
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.49 sec. Users per second: 865
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=

In [24]:
list_recommender = [ucf, rp3b, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo18 = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.9659118532652889, 'l1_ratio': 0.8328907910906823}

    combo18.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo18)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.03 sec. Users per second: 807
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 6.89 sec. Users per second: 815
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
UserKNNCFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender loaded. [seed=1666,

In [25]:
list_recommender = [icb, icf, icfcb]
result_list = []
for i in range(len(URM_list)):
    combo19= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.07009442547942947, 'l1_ratio': 0.6748980281927442}

    combo19.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo19)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.70 sec. Users per second: 736
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.66 sec. Users per second: 733
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Reco

In [26]:
list_recommender = [icb, icf, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo20= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.7812428236392996, 'l1_ratio': 0.14931685498693983}

    combo20.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo20)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
ItemKNNCFRecommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.74 sec. Users per second: 732
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
ItemKNNCFRecommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.63 sec. Users per second: 736
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
ItemKNNCFRecommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender loaded. [s

In [27]:
list_recommender = [icb, p3a, icfcb]
result_list = []
for i in range(len(URM_list)):
    combo21= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.3589679390960739, 'l1_ratio': 0.19013969440122908}

    combo21.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo21)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
P3alphaRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.61 sec. Users per second: 745
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
P3alphaRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 8.07 sec. Users per second: 696
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
P3alphaRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommende

In [28]:
list_recommender = [icb, rp3b, icfcb]
result_list = []
for i in range(len(URM_list)):
    combo22= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.08269957561423759, 'l1_ratio': 0.34697712734236047}

    combo22.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo22)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.10 sec. Users per second: 799
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
ItemKNN_CBF_CF_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.53 sec. Users per second: 745
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666, fold=2]
ItemKNN_CBF_CF_Recommende

In [ ]:
list_recommender = [icb, rp3b, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo23= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.6117345841439186, 'l1_ratio': 0.2677959085246144}

    combo23.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo23)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

HybridCombinationSearchCVtest: URM Detected 48 (0.60 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2381 (9.17 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=0]
RP3betaRecommender loaded. [seed=1666, fold=0]
SLIM_BPR_Recommender loaded. [seed=1666, fold=0]
EvaluatorHoldout: Processed 5667 ( 100.00% ) in 7.74 sec. Users per second: 732
HybridCombinationSearchCVtest: URM Detected 68 (0.86 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2439 (9.39 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=1]
RP3betaRecommender loaded. [seed=1666, fold=1]
SLIM_BPR_Recommender loaded. [seed=1666, fold=1]
EvaluatorHoldout: Processed 5616 ( 100.00% ) in 7.10 sec. Users per second: 790
HybridCombinationSearchCVtest: URM Detected 56 (0.70 %) cold users.
HybridCombinationSearchCVtest: URM Detected 2402 (9.25 %) cold items.
ItemKNNCBFRecommender loaded. [seed=1666, fold=2]
RP3betaRecommender loaded. [seed=1666, fold=2]
SLIM_BPR_Recommender loaded. [seed=16

In [ ]:
list_recommender = [icb, ucf, icfcb]
result_list = []
for i in range(len(URM_list)):
    combo24= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params= {'alpha': 0.3222784627089323, 'l1_ratio': 0.13265583683303742}

    combo24.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo24)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

In [ ]:
"""
list_recommender = [icb, ucf, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo25= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params=
    combo25.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo25)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))
"""

In [ ]:
list_recommender = [icb, p3a, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo26= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params={'alpha': 0.9970595877880616, 'l1_ratio': 0.16423348901694348}

    combo26.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo26)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

In [ ]:
list_recommender = [icb, icfcb, sbpr]
result_list = []
for i in range(len(URM_list)):
    combo27= HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_recommender, seed = seed, fold = i)
    params= {'alpha': 0.9366901381834416, 'l1_ratio': 0.07977626900958935}

    combo27.fit(**params)
    result, _ = evaluator_list[i].evaluateRecommender(combo27)
    result_list.append(result[10]['MAP'])
print(sum(result_list)/len(result_list))

---
PLOT

In [ ]:
list_rec = [([icb, icf, p3a], {'alpha': 0.6795644006301982, 'l1_ratio': 0.16897463528047377}),
            ([icb, icf, rp3b], {'alpha': 0.17669196327123107, 'l1_ratio': 0.3076285474940705}),
            ([icb, icf, ucf], {'alpha': 0.6795644006301982, 'l1_ratio': 0.16897463528047377}),
            ([icb, rp3b, p3a], {'alpha': 0.3003711591677414, 'l1_ratio': 0.520142870843707}),
            ([icb, ucf, p3a], {'alpha': 0.5989306837162718, 'l1_ratio': 0.19159245289609933}),
            ([icb, ucf, rp3b], {'alpha': 0.22386976323299096, 'l1_ratio': 0.5121960118838208}),
            ([icf, ucf, p3a], {'alpha': 0.176675887233281, 'l1_ratio': 0.11869923626931214}),
            ([icf, ucf, rp3b], {'alpha': 0.961794201569176, 'l1_ratio': 0.6153004656692367}),
            ([p3a, icfcb, sbpr], {'alpha': 0.9901471939440843, 'l1_ratio': 0.004332872503698072}),
            ([rp3b, icfcb, sbpr], {'alpha': 0.9929867621463604, 'l1_ratio': 0.009578827985342044}),
            ([rp3b, p3a, icfcb], {'alpha': 0.05856975370672747, 'l1_ratio': 0.4436754753867742}),
            ([rp3b, p3a, sbpr], {'alpha': 0.9905861627687869, 'l1_ratio': 0.0009938396703816912}),
            ([ucf, icfcb, sbpr], {'alpha': 0.9914207032100951, 'l1_ratio': 0.007542874998078932}),
            ([ucf, p3a, icfcb], {'alpha': 0.016697522704458127, 'l1_ratio': 0.3696380999702476}),
            ([ucf, p3a, sbpr], {'alpha': 0.9933589553636074, 'l1_ratio': 0.0060490313929022505}),
            ([ucf, rp3b, icfcb], {'alpha': 0.016697522704458127, 'l1_ratio': 0.3696380999702476}),
            ([ucf, rp3b, p3a], {'alpha': 0.014042798897581758, 'l1_ratio': 3.1039843699565146e-05}),
            ([ucf, rp3b, sbpr], {'alpha': 0.9659118532652889, 'l1_ratio': 0.8328907910906823}),
            ([icb,icf,icfcb], {'alpha': 0.07009442547942947, 'l1_ratio': 0.6748980281927442}),
            ([icb,icf,sbpr], {'alpha': 0.7812428236392996, 'l1_ratio': 0.14931685498693983}),
            ([icb,p3a,icfcb], {'alpha': 0.3589679390960739, 'l1_ratio': 0.19013969440122908}),
            ([icb,rp3b,icfcb], {'alpha': 0.08269957561423759, 'l1_ratio': 0.34697712734236047}),
            ([icb,rp3b,sbpr], {'alpha': 0.6117345841439186, 'l1_ratio': 0.2677959085246144}),
            ([icb,ucf,icfcb], {'alpha': 0.3222784627089323, 'l1_ratio': 0.13265583683303742}),
            #([icb, ucf, sbpr], ),
            ([icb, p3a, sbpr], {'alpha': 0.9970595877880616, 'l1_ratio': 0.16423348901694348}),
            ([icb, icfcb, sbpr], {'alpha': 0.9366901381834416, 'l1_ratio': 0.07977626900958935}),
]

In [ ]:
num_combo = len(list_rec)

In [ ]:
f_ranges = [
    (0,3),
    (3,5),
    (5,10),
    (10,17),
    (17,30),
    (30,100),
    (100,200),
    (200,-1)
]

MAP_combo_per_group = []
cutoff = 10

for f_range in f_ranges:
    URM_test_group = parser.filter_URM_test_by_range(URM_train, URM_test, f_range)
    evaluator_test = EvaluatorHoldout(URM_test_group, cutoff_list=[cutoff])
    
    combo_dict={}
    for k in range (num_combo):
        r = []
        for i in range (len(URM_list)):
            rec = HybridCombinationSearchCVtest(URM_list[i], ICM_all, list_rec[k][0], seed = seed, fold = i)
            rec.fit(**list_rec[k][1])

            results, _ = evaluator_list[i].evaluateRecommender(rec)
            r.append(results[cutoff]["MAP"])
        combo_dict[k]=np.average(r)

    MAP_combo_per_group.append(combo_dict)

In [ ]:
def check_if_present(key_list, el):
    for key_val, combo_val in key_list:
        if key_val == el[0] and combo_val == el[1]:
            return True
    return False

In [ ]:
to_be_plotted = []
for i in range(0, len (f_ranges)):
    print(f"RANGE {f_ranges[i]}")
    maxp = 0
    difference = 0
    old_name = ''
    max_name=''
    max_isCombo=None
    old_isCombo=None
    max_key=None
    old_key=None
    
    combo_dict = MAP_combo_per_group[i]
    for key in combo_dict:
        value = combo_dict[key]
        print(f"> MAP combo-{combo_labels[key]}:\t {value}")
        if value > maxp:
            difference=value - maxp
            maxp=value
            old_name=max_name
            old_isCombo=max_isCombo
            max_name=combo_labels[key]
            max_isCombo=True
            old_key=max_key
            max_key=key
           
    if not check_if_present(to_be_plotted, (max_key, max_isCombo)):
        to_be_plotted.append((max_key, max_isCombo))
    
    if not old_key is None and not check_if_present(to_be_plotted, (old_key, old_isCombo)):
        to_be_plotted.append((old_key, old_isCombo))
    
    
    print(f"Max MAP is {maxp} from {max_name}({'COMBO' if max_isCombo else 'MERGED'}) with difference from previous max of {difference} from {old_name}({'COMBO' if old_isCombo else 'MERGED'}) \n")
        

In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline  
pyplot.figure(figsize=(20,20))
for key, isCombo in to_be_plotted:
    l=[]
    if isCombo:
        for combo_dict in MAP_combo_per_group:
            l.append(combo_dict[key])
        style = '--o'
        label = combo_labels[key]
    """
    else:
        for merged_dict in MAP_merged_per_group:
            l.append(merged_dict[key])
        style = '--x'
        label = merged_labels[key]
    pyplot.plot(l, style, label = label)
    """

pyplot.xticks(ticks=range(0,len(f_ranges)), labels=f_ranges)
pyplot.ylabel('MAP')
pyplot.xlabel('User Group')
pyplot.legend()
pyplot.show()